<a href="https://colab.research.google.com/github/sinnarasam/StudyJAM/blob/main/intro_prompt_design_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [ ]:
### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

SyntaxError: invalid syntax (1258973391.py, line 3)

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "qwiklabs-gcp-00-a241abe91196"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [ ]:
model = GenerativeModel("gemini-1.5-flash")

In [ ]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [ ]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried flowers, playing with different themes:

**Whimsical & Romantic:**

* The Dried Petal
* Bloom & Wither
* Everlasting Blooms
* The Dried Flower Garden
* Whispers of Wildflowers
* Timeless Blooms

**Modern & Minimalist:**

* The Dried Bouquet
* Bloom & Fade
* Faded Beauty
* Paper & Petal
* Botanical Hues
* Bloom & Co.

**Unique & Playful:**

* The Dusty Bouquet
* The Sun-Kissed Bloom
* The Herbarium
* The Flower Press
* Forever Blooms
* The Dried Flower Nook

**Location-based:**

* [Your City] Dried Flowers
* The [Your Street Name] Flower Co.
* [Your Neighborhood] Bloomery

**Tips for Choosing a Name:**

* **Keep it memorable and easy to pronounce.**
* **Reflect your brand and target audience.**
* **Check for availability of the name and domain name.**
* **Consider your business's location and branding.**

Ultimately, the best name for your shop will be one that you love and that resonates with your customers. Good luck! 



✅ Recommended. The prompt below is to the point and concise.

In [ ]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

Here are some names for a flower shop that sells bouquets of dried flowers, playing on different themes:

**Romantic & Timeless:**

* **Everlasting Bloom**
* **Dried & Delicate**
* **Bloom & Blossom**
* **The Wilted Rose**
* **Whispers of Bloom**
* **The Dried Petal**

**Modern & Trendy:**

* **Botanica Dried**
* **Bloom & Preserve**
* **Dried Blooms Co.**
* **The Bloom Bar**
* **Everlasting Flora**

**Whimsical & Unique:**

* **The Flower Alchemist**
* **Paper & Petals**
* **The Botanical Muse**
* **Dusty Blooms**
* **Wildflower & Co.**

**Location-Specific:**

* [Your City] Dried Flowers
* [Your Neighborhood] Bloom
* [Your Street] Petals

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Consider your target audience.**
* **Make sure the name is available as a domain name and social media handle.**
* **Get feedback from friends and family.**

Remember to choose a name that you love and that reflects the unique style of your shop. Good luck! 



### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [ ]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known place in the universe where life is known to exist. It's a vibrant and dynamic world, constantly evolving and changing. 

Here are some key facts about Earth:

**Physical Characteristics:**

* **Diameter:** 12,742 kilometers (7,926 miles)
* **Surface area:** 510 million square kilometers (197 million square miles)
* **Mass:** 5.972 × 10^24 kg
* **Average density:** 5.52 g/cm³
* **Surface gravity:** 9.81 m/s²
* **Rotation period:** 23 hours, 56 minutes, 4 seconds
* **Orbital period:** 365.25 days
* **Average distance from the Sun:** 149.6 million kilometers (93 million miles)

**Composition:**

* **Atmosphere:** Primarily nitrogen (78%), oxygen (21%), and other trace gases.
* **Surface:** Composed of 71% water (oceans, lakes, rivers) and 29% land (continents, islands).
* **Interior:**  A core (solid inner core and liquid outer core), mantle (solid but flowing), and crust (the thin outer layer).

**Life:

✅ Recommended. The prompt below is specific and well-defined.

In [ ]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## Earth's Unique Traits: What Makes Our Planet Special

Earth stands out in the cosmic landscape due to a fascinating combination of factors, making it the only known abode of life as we know it. Here's a list of key features that make our planet unique:

**1. Liquid Water:** Earth boasts vast oceans covering 71% of its surface. This liquid water is essential for life as we know it, serving as a solvent, a medium for transporting nutrients, and a key component of biological processes.

**2. Plate Tectonics:** Earth's crust is divided into plates that move and interact, shaping the planet's continents, mountains, and oceans. This dynamic process helps regulate the planet's climate and provides a unique geological history.

**3. Atmosphere:** Earth's atmosphere, rich in nitrogen and oxygen, provides a protective shield against harmful solar radiation. It also helps regulate temperature through the greenhouse effect, making life possible.

**4. Magnetic Field:** Earth's core generates a 

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [ ]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's tackle these two questions separately:

**Boiling Water**

The **best method** depends on what you're using it for:

* **Fastest:** A stovetop kettle or electric kettle is usually the fastest way to boil water for everyday tasks like tea or instant coffee.
* **Most energy efficient:** A stovetop kettle with a tight-fitting lid is generally the most energy efficient as it requires less energy to heat the water.
* **Safest:** A microwave is the safest method for boiling small amounts of water, especially for people with limited mobility. 
* **Large quantities:** A stovetop pot with a wide base is best for boiling large quantities of water for cooking pasta or other dishes.

**Why is the Sky Blue?**

The blue color of the sky is caused by a phenomenon called **Rayleigh scattering**.  Here's how it works:

1. **Sunlight:** Sunlight is made up of all colors of the rainbow.
2. **Atmosphere:** As sunlight enters Earth's atmosphere, it encounters air molecules (mainly nitrogen and oxygen

✅ Recommended. The prompts below asks one task a time.

In [ ]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

The best method for boiling water depends on your needs and resources. Here's a breakdown:

**For speed and convenience:**

* **Electric kettle:** This is the fastest and most convenient way to boil water. Modern kettles are also energy-efficient and have safety features like automatic shut-off.

**For versatility and control:**

* **Stovetop:**  You can use a pot or saucepan on your stovetop. This gives you more control over the heat and can be used for larger volumes of water.

**For travel or camping:**

* **Camping stove:** These are lightweight and portable, allowing you to boil water in the outdoors.
* **Microwave:** This is quick and easy, but not ideal for larger volumes of water.

**Here's what to consider when choosing a method:**

* **Volume of water:** Electric kettles are best for small to medium volumes, while stovetop methods are more versatile for larger volumes.
* **Time constraints:** Electric kettles and microwaves are the fastest options.
* **Energy efficiency:** El

In [ ]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is made up of all colors of the rainbow.**  These colors have different wavelengths, with blue light having a shorter wavelength than red light.
* **When sunlight enters the Earth's atmosphere, it interacts with the tiny molecules of air.** These molecules are much smaller than the wavelength of visible light.
* **This interaction causes the light to scatter in all directions.**  This scattering is more pronounced for shorter wavelengths like blue and violet.
* **Blue light is scattered more effectively than other colors.**  This is because the shorter wavelength of blue light interacts more strongly with the air molecules.
* **As a result, we see a blue sky because the scattered blue light reaches our eyes from all directions.**

**Think of it like this:** Imagine a tiny dust particle in the air. When sunlight hits it, the dust particle scatters the light. Blue light scatters mo

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [ ]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is today, please check a calendar or ask a voice assistant. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [ ]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan offers a wealth of sightseeing opportunities! To give you the best suggestion, I need a little more information. What are you most interested in seeing? 

* **Art and History:** Do you want to visit iconic landmarks like the Duomo or explore Renaissance art at the Pinacoteca di Brera?
* **Fashion and Design:** Are you interested in the world of fashion and design, perhaps visiting the Museo del Novecento or the Armani/Silos museum?
* **Food and Culture:** Do you want to immerse yourself in Milanese culture, trying local cuisine and exploring vibrant markets?

Tell me more about your interests, and I can help you discover the perfect sightseeing spots in Milan! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [ ]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers plan their trips, and that includes providing information about sightseeing, accommodations, and transportation.  I'm not equipped to give personal advice like meal recommendations.  

Perhaps you could try a search engine or a food delivery app to find some delicious dinner options! 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

## Programming Activity Recommendations for High School Students:

**Beginner (Just starting out):**

* **Build a Simple Calculator:**  Write a program that takes two numbers as input and performs basic operations like addition, subtraction, multiplication, and division.
* **Create a Text-Based Adventure Game:**  Design a simple story with choices, using conditional statements and loops to guide the player's path.
* **Develop a To-Do List App:**  Use lists and data structures to store tasks, and allow users to add, delete, and mark tasks as complete.

**Intermediate (Some experience, familiar with basic concepts):**

* **Build a Hangman Game:**  Implement the classic game, including logic for guessing letters, displaying the word, and keeping track of attempts.
* **Create a Simple Quiz Application:**  Design a quiz with multiple-choice questions, store answers, and provide feedback at the end.
* **Develop a Weather App:**  Use an API to fetch weather data for a location and display it 

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

It's great you're exploring programming!  All three languages have value, but the best choice depends on your interests and goals:

**a) Learn Python:**

* **Why:** Python is highly versatile and beginner-friendly.  It's used in:
    * **Data science and machine learning:** Libraries like NumPy, Pandas, and scikit-learn make Python ideal for analyzing data and building AI models.
    * **Web development:** Frameworks like Django and Flask are popular for building websites.
    * **Scripting and automation:** Easy to automate tasks, making it great for personal projects.
    * **Game development:** Libraries like Pygame allow you to create simple games.
* **Pros:** Large community, excellent documentation, vast libraries, easy to learn syntax.
* **Cons:** Can be slower than other languages for certain tasks.

**b) Learn JavaScript:**

* **Why:** JavaScript is the language of the web. It's used to:
    * **Interactive websites:** Add dynamic elements like animations, forms, and user inte

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 

Here's why:

* **"Something surprised me"** implies a positive experience.
* **"It was actually original"** is a strong compliment. 
* **"Not the same old recycled stuff"**  further highlights the positive aspect of originality.
* **"Watch it - you will not regret it"** is a clear recommendation, suggesting a positive experience for others. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.